# Prerequisites

This notebook requires the hnxwidget package; please install by running:

```pip install hnxwidget jupyter_contrib_nbextensions jupyter_nbextensions_configurator```

# HNX Constructor and Widget Examples

Unlike the tutorials, this is an interactive demo to get you acquainted with the constructor options and how to use the widget. **Hover over the nodes and edges each time you run the widget to see how properties enhance the visual information.**

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import networkx as nx
import hypernetx as hnx
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter('ignore')

try:
    from hnxwidget import HypernetxWidget as HW
except:
    print("Required dependencies not installed. To install, please run: pip install hnxwidget jupyter_contrib_nbextensions jupyter_nbextensions_configurator")


In [ ]:
def checkplts(h):
    fig,ax = plt.subplots(1,2,figsize=(15,6))
    hnx.draw(h,ax=ax[0])
    ax[0].set_title('Hypergraph',fontsize=15)
    hnx.draw(h.dual(),ax=ax[1])
    ax[1].set_title('Dual',fontsize=15)


## Set Systems 
The next 2 cells construct the necessary dictionaries and dataframes to run the demo.

In [ ]:
### numpy array, single property dict - uncomment np.random.seed for consistent results
# np.random.seed(0)
npcol1 = np.random.choice(list("ABCD"),50)
npcol2 = np.random.choice(list("abcdefghijklmnopqrstuvwxyz"),50)

npdata = np.array([npcol1,npcol2]).T
npedge_col = 'Club'
npnode_col = 'Member'

npproperties = {k :{'affiliation': np.random.choice(['red','green'])} for k in np.concatenate([npcol1,npcol2])}

In [ ]:
## LesMis data as dictionaries and dataframes - uses LesMis class in utils.toys
## Uncomment np.random.seed for consistent results
# np.random.seed(0)
from hypernetx.utils.toys import lesmis as lm
np.random.seed(0)
LM = lm.LesMis()
## dict
scenes = {
    0: ('FN', 'TH'),
    1: ('TH', 'JV'),
    2: ('BM', 'FN', 'JA'),
    3: ('JV', 'JU', 'CH', 'BM'),
    4: ('JU', 'CH', 'BR', 'CN', 'CC', 'JV', 'BM'),
    5: ('TH', 'GP'),
    6: ('GP', 'MP'),
    7: ('MA', 'GP'),
}

### Nested dict with cell_properties
scenes_with_cellprops = {ed: {ch: {'color':np.random.choice(['red','green']),'cell_weight':np.random.rand()} 
                         for ch in v} for ed,v in scenes.items()}

### Pandas dataframe
scenes_df = pd.DataFrame(pd.Series(scenes).explode()).reset_index().rename(columns={'index':'Scenes', 
                                                                                           0:'Characters'})
### Dataframe with cell properties
scenes_dataframe = scenes_df.copy()
scenes_dataframe['color'] = np.random.choice(['red','green'],len(scenes_dataframe))
scenes_dataframe['heaviness'] = np.random.rand(len(scenes_dataframe))


### Node and edge property data
nodes = list(set(list(np.concatenate([v for v in scenes.values()]))))
edges = list(set(list(scenes.keys())))
node_properties = {ch: {'FullName': LM.dnames.loc[ch].FullName,                               
                        'Description': LM.dnames.loc[ch].Description,
                       'color':np.random.choice(['pink','lightblue'])} for ch in nodes}
node_props_df = pd.DataFrame.from_dict(node_properties,orient='index')
default_node_weight = 10

### These edge properties will have missing weights so 
### will be filled by constructor with default_edge_weight
edge_properties = defaultdict(dict)
edge_properties.update({ed:{'weight':np.random.randint(2,10)} for ed in range(0,8,2)})
for ed in edges:
    edge_properties[ed].update({'color':np.random.choice(['red','green'])})
default_edge_weight = 2

properties = [{'id':nd,
               'color':np.random.choice(['red','blue','green','yellow']),
               'weight': np.round(np.random.rand(),3)}
              for nd in nodes+edges]
properties = pd.DataFrame(properties)


## Hypergraphs without properties

In [ ]:
def test(H):
    edge = list(H.edges)[0]
    node = H.edges[edge][0]
    pair = (edge,node)
    return {'pair' : pair,
            'nodes' : list(H.nodes)[:5],
            'edges' : list(H.edges)[:5],
            'diameter': H.diameter(),
            'edge_diameter' : H.edge_diameter(),
            'linegraph': H.get_linegraph(1).edges(), 
            'info' : hnx.info_dict(H),
            'get_cell_property' : H.edges.get_cell_properties(edge,node)}


#### Numpy Arrays

In [ ]:
### With no data and dictionaries, constructor works as before but will now accept 
### n x 2 dimensional Numpy ndarrays.
H1 = hnx.Hypergraph(npdata)
test(H1)

In [ ]:
checkplts(H1)

#### Pandas DataFrames

In [ ]:
scenes_dataframe[:5]

In [ ]:
## dataframes will by default use the first two columns for (edge,node) pairs 
### but different columns may be specified using the edge_col and node_col keywords
H2 = hnx.Hypergraph(scenes_dataframe)
checkplts(H2)

In [ ]:
H2 = hnx.Hypergraph(scenes_dataframe,edge_col='Characters',node_col='Scenes')
checkplts(H2)

## Hypergraphs from setsytems with cell properties

In [ ]:
def testp(H):
    edge = list(H.edges)[0]
    node = H.edges[edge][0]
    pair = (edge,node)
    HD = H.dual()
    return {
            'pair' : pair,
            'single_cell_property' : H.get_cell_properties(edge,node),
            'single_cell_weight' : H.get_cell_properties(edge,node,H._cell_weight_col),
            'single_dual_cell_property' : HD.get_cell_properties(node,edge),
            'neighbors' : H.neighbors(node),
            'edge_neighbors': H.edge_neighbors(edge),
            'line_graph': H.get_linegraph(edges=True)
            }

### Dataframes with properties

In [ ]:
H3 = hnx.Hypergraph(scenes_dataframe,
                   cell_properties=['color'],
                   cell_weight_col='heaviness',
                  node_properties=node_properties,
                  edge_properties=edge_properties)
testp(H3)

In [ ]:
H3.incidence_matrix(weights=True).todense()

### Add object properties
Hover over nodes and edges in the widget to see their properties

In [ ]:
H4 = hnx.Hypergraph(scenes_dataframe,
                   cell_properties=['color'],
                   cell_weight_col='heaviness',
                  properties=properties,
                  weight_col='weight')

In [ ]:
HW(H4)

In [ ]:
H5 = H.remove(['JV',1,2,3])
HW(H5)

In [ ]:
### Line Graphs persist properties as well

In [ ]:
h = H4 ## Try with H3
G1 = h.get_linegraph()
G2 = h.get_linegraph(edges=False)
nxncolors = [h.nodes[nd].color for nd in G2.nodes]
nxecolors = [h.edges[nd].color for nd in G1.nodes]
fig,ax = plt.subplots(1,2,figsize=(15,7))
nx.draw_networkx(G1,node_color = nxecolors,ax=ax[0])
ax[0].set_title('edge line graph',fontsize=15)
ax[0].axis('off')
nx.draw_networkx(G2,node_color = nxncolors,ax=ax[1])
ax[1].set_title('node line graph',fontsize=15)
ax[1].axis('off');

In [ ]:
G1.nodes(data=True)

In [ ]:
G2.nodes(data=True)

### Dictionaries with properties

In [ ]:
H5 = hnx.Hypergraph(scenes_with_cellprops,
                   edge_col='Scenes',
                   node_col='Characters',
                   cell_weight_col = 'cell_weight', 
                   cell_properties=scenes_with_cellprops)
testp(H.dual())

In [ ]:
checkplts(H5.collapse_nodes())

In [ ]:
H5.incidence_matrix(weights=True).todense()


In [ ]:
H5.adjacency_matrix(s=2).todense()

In [ ]:
H5.edge_adjacency_matrix().todense()

In [ ]:
H5.get_cell_properties(0,'FN','color')

## Hypergraphs with properties on edges and nodes

In [ ]:
H6 = hnx.Hypergraph(
    setsystem=scenes_dataframe,
    edge_col="Scenes",
    node_col="Characters",
    cell_weight_col='cell_weight',
    cell_properties=['color'],
    edge_properties=edge_properties,
    node_properties=node_properties,
    default_edge_weight=2.5,
    default_node_weight=6,
)
H6.properties

In [ ]:
plot = HW(H6,node_fill = {nd:H6.nodes[nd].color for nd in H6.nodes},
         edge_stroke = {ed:H6.edges[ed].color for ed in H6.edges},
         edge_stroke_width = {ed:12 for ed in H6.edges})
plot

In [ ]:
### Properties are preserved when removing or restricting edges or taking toplexes
tops = H6.toplexes()
HW(tops)

In [ ]:
tops.edges.properties

### np array with node and edge data

In [ ]:
npproperties

In [ ]:
H7 = hnx.Hypergraph(npdata,
                    edge_col=npedge_col,
                    node_col=npnode_col,
                    properties = npproperties)
HW(H7)

In [ ]:
HW(H7,node_fill={nd: H7.nodes[nd].affiliation for nd in H7.nodes},
      edge_stroke={ed: H7.edges[ed].affiliation for ed in H7.edges})

## Hypergraphs with multi-edges
HNX distinguishes between edges by their ids, not their contents. This allows for multi-edges


In [ ]:
df1 = scenes_df.copy()
df1['cell_weights'] = 1
df2 = scenes_df.copy()
df2.Scenes = df2.Scenes.apply(lambda x : str((x + 8)))

## Duplicate edges
ndf = pd.concat([df1,df2])
## Change an attribute on duplicate edges to try aggregation methods
ndf['color'] = np.random.choice(['red','lightblue','green'],len(ndf))

In [ ]:
H8 = hnx.Hypergraph(ndf)
HW(H8)

In [ ]:
H9,eclasses = H8.collapse_edges(return_equivalence_classes=True)
## equivalence classes for collapsed edges
eclasses

In [ ]:
HW(H9)

## Restrict_to and Remove 

The same restriction can be used for remove nodes and restrict_to methods. Depending on the number of objects being restricted to it could be faster to just remove the objects you don't want.

In [ ]:
H10 = hnx.Hypergraph(scenes_dataframe,node_properties=node_properties,edge_properties=edge_properties)
HW(H10)

In [ ]:
HW(H10.restrict_to_nodes(['JV','TH','BM','FN']))

In [ ]:
HW(H10.restrict_to_edges([0,1]))

In [ ]:
HW(H10.remove_edges([0,1]))

In [ ]:
### If nodes and edges have distinct id sets a single remove command cat remove both.
HW(H10.remove(['JV','TH',2,3]))
